In [1]:
using Gridap
using Optim

In [2]:
## Define solutions ##

u(x) = 11*x[1]^2 - 0.3*x[2]^4  + 5*sin(x[1]) - 3*x[1]*cos(x[2]) + exp(x[2])

u (generic function with 1 method)

In [3]:
## Construct geometry ##

domain = (0,3,0,1)
n = 30
partition = (3n,n)
model = CartesianDiscreteModel(domain,partition)
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

In [4]:
## Define Test and Trial Spaces ##

order = 2
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V0,u)

TrialFESpace()

In [5]:
## This ##

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [6]:
## Solve PDE ##

function forward(p)
    f(x) = p[1] + p[2]*x[2]^2  + p[3]*sin(x[1]) + p[4]*x[1]*cos(x[2]) + p[5]*exp(x[2])
    a(u,v) = ∫(∇(v)⋅∇(u))*dΩ
    b(v)   = ∫(v*(f))*dΩ
    op     = AffineFEOperator(a,b,U,V0)
    uh     = solve(op)
    return uh
  end


forward (generic function with 1 method)

In [ ]:
##  Objective Function ##

function J(p)
  uh = forward(p)
  e = u - uh
  el2 = sqrt(sum( ∫( e*e )*dΩ ))
  return el2
end

J (generic function with 1 method)

In [13]:
## Optimization ##

using Zygote, ADTypes, ReverseDiff, ForwardDiff, Enzyme
  
opts = Optim.Options(
  g_tol    = 1.0e-4,   # stop when ‖∇J‖ ≤ 1e-4
  iterations = 20,  # max number of iterations
  show_trace = true  # (optional) print progress
)

p0 = [1.0, 1.0, 1.0, 1.0, 1.0]

res = optimize(
  J,
  p0,
  BFGS(),
  # autodiff = Zygote.AutoDiff(), # doesn't work
  opts,
)

println("Estimated p = ", Optim.minimizer(res))
println("True p = -22, 3.6, 5, -3, -1")

Iter     Function value   Gradient norm 
     0     3.619845e+00     2.198626e-01
 * time: 0.0
     1     1.665885e+00     2.153731e-01
 * time: 5.364000082015991
     2     4.008445e-01     1.713217e-01
 * time: 7.254000186920166
     3     3.417825e-01     1.712487e-01
 * time: 10.912000179290771
     4     2.304553e-01     2.238969e-02
 * time: 12.578999996185303
     5     2.291208e-01     3.619964e-02
 * time: 15.338000059127808
     6     1.905935e-01     1.517054e-02
 * time: 21.005000114440918
     7     1.883685e-01     8.288979e-03
 * time: 23.151000022888184
     8     1.340974e-01     2.519460e-02
 * time: 30.514000177383423
     9     9.666872e-02     7.991001e-03
 * time: 32.218000173568726
    10     9.639524e-02     1.027290e-02
 * time: 34.009000062942505
    11     9.597676e-02     1.905162e-02
 * time: 36.746999979019165
    12     7.350697e-02     2.705388e-02
 * time: 42.09400010108948
    13     1.852336e-02     2.996217e-02
 * time: 44.18500018119812
    14     9